In [54]:
import sqlite3
import glob
import urllib.parse
import records
import requests
from bs4 import BeautifulSoup
import MeCab as mc
import re
from collections import Counter
from matplotlib import pyplot as plt
import gensim
from gensim.models import Word2Vec
from gensim import matutils, corpora
from gensim.corpora.dictionary import Dictionary

In [10]:
conn = None

# データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect("./yahoo_all2.db")
    
# コネクションを断つ
def close():
    conn.close()
    
# テーブル作成
def create_table():
    # DROP=消す.
    conn.execute("DROP TABLE IF EXISTS Yahoo_s")
    conn.execute("""CREATE TABLE Yahoo_s (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            category TEXT
    )""")
    
# loadは積むという意味
def load(j,category):
    conn.execute(
        "INSERT INTO Yahoo_s (title,category) VALUES (?,?)", (j,category))
    conn.commit()
    


In [3]:
url = 'https://news.yahoo.co.jp'

r = requests.get(url)
html_soup = BeautifulSoup(r.text, 'html.parser')
# print(html_soup)

ul = html_soup.find("ul", class_ ="yjnHeader_sub_cat")
it = ul.find("a", text = "IT")
sports = ul.find("a", text = "スポーツ")
business = ul.find("a", text = "経済")
science = ul.find("a", text = "科学")
# print(it["href"])
# print(sports["href"])

In [4]:
r_it = requests.get(url+it["href"])
r_sports = requests.get(url+sports["href"])
r_business = requests.get(url+business["href"])
r_science = requests.get(url+science["href"])
it_soup = BeautifulSoup(r_it.text, 'html.parser')
sports_soup = BeautifulSoup(r_sports.text, 'html.parser')
business_soup = BeautifulSoup(r_business.text, 'html.parser')
science_soup = BeautifulSoup(r_science.text, 'html.parser')

In [12]:
connect()
create_table()
close()

In [11]:
def title_load(category):
    try:
        connect()
        for i in range(100):
            all_it = requests.get(it_more["href"]+"&p="+str(i+1))
            for_it = BeautifulSoup(all_it.text, 'html.parser')
            list_it = [i.find("dt").text for i in for_it.find_all("dl", class_="title")]
            for j in list_it:
                load(j,category)
    finally:
        close()

In [13]:
it_more = it_soup.find("a", text = "もっと見る")
title_load("IT")

In [14]:
sports_more = sports_soup.find("a", text = "もっと見る")
title_load("Sports")

In [15]:
business_more = business_soup.find("a", text = "もっと見る")
title_load("Business")

In [16]:
science_more = science_soup.find("a", text = "もっと見る")
title_load("Science")

In [37]:
try:
    connect()
    cursor = conn.cursor()
    res_news = cursor.execute("SELECT title FROM Yahoo_s")
    res_news = cursor.fetchall()
finally:
    close()

In [ ]:
stop_words = ["ゅ","～","?"]

In [38]:
res_list = [mecab_analysis(r[0]) for r in res_news]

In [22]:
res_list

[['無限', 'アラート', '2', '人', '起訴', '処分'],
 ['TDS', '噴水', '足', 'の', 'NG'],
 ['ポケモン', '睡眠', 'エン', 'タメ', '化'],
 ['ポケモン', '新作', 'ゲーム', '2', '本', '発表'],
 ['ポケモン', 'DeNA', '新作', 'ゲーム'],
 ['海外', '共感', '?', 'スマホ', 'NG', '結婚式'],
 ['iPod', 'touch', 'ゲーム', '重点'],
 ['新型', 'iPod', 'touch', '発表'],
 ['スマホ', '商戦', '異状', '上位', '逆風'],
 ['ビット', 'コイン', '価格', '回復', '背景'],
 ['見城', '氏', '日本国', '紀', '批判', '一蹴'],
 ['林', '死刑', '囚', '長男', '発信', '理由'],
 ['賞', '贈呈', '式', '作家', '幻冬舎', '批判'],
 ['傘', 'コラ', '画像', 'デマ', '拡散'],
 ['大統領', '握手', '作家', '招待', '否定'],
 ['外資', '規制', 'IT', '情報', '流出'],
 ['配膳', 'ロボット', '導入', '現場', '声'],
 ['ハムスター', '動画', '虐待'],
 ['米', '華', '為', '排除', 'もろ刃', '剣'],
 ['商談', '良しあし', 'ソフト', '判定'],
 ['不謹慎', '不毛', '連鎖'],
 ['華', '為', 'スマホ', '出荷', '最大', '24', '%', '減'],
 ['パオ', 'チャン', '活動', '休止'],
 ['SNS', '炎上', '防災', '訓練', '人気'],
 ['安全ピン', '痴漢', '撃退', '過剰', '?', '賛否'],
 ['在宅', 'トラブル', '注意'],
 ['荒野', '行動', 'PUBG', '和解', '成立'],
 ['NGT', '支配人', 'Twitter', '開始', '賛否'],
 ['FB', 'アカ', '半年', '30', '億', '件', '削除'],


In [80]:
text = "".join(res_list)
print(text)

平尾誠二さん 娘明かす亡き父U20日本 大人のサッカー必要大谷 MLB球宴DHで候補入りももクロ マー君初観戦に興奮松坂 来月21日にも1軍復帰か大坂初戦突破 人生で一番緊張全仏 大坂が大逆転で初戦突破幻の80年五輪代表 笹田氏死去全仏OP 大坂なおみの1回戦八百長 元レアルDFらを逮捕浦和監督解任 後任は「組長」羽生結弦 見せた確かな回復川内優輝 世界陸上で代表復帰青木宣親丸刈り 体張って鼓舞中日松坂 257日ぶり実戦登板188cmなのに下手投げ SB右腕相撲で座布団投げ 罪になる?大坂「開き直り」で赤土克服大谷が技あり安打 二盗は失敗五輪チケ申し込み きょうまでドラフト 指名優先権を変更へ2軍戦で100球 輝星昇格へ条件横審 混乱招いた審判部に苦言全豪準Vのクビトバ 全仏棄権広島浮上 きっかけは丸2ラン広島浮上 4月の丸2ランで覚醒夏場所に大統領 視聴率26.3%単勝93倍 伏兵なぜダービーV朝乃山に親方 クーラー買えよ何もやってない 挑む本田圭佑マエケン先発復帰 今季6勝目U20日本 メキシコに3発快勝Jで懐かしのビスマルクポーズ全仏OP 錦織圭が初戦突破FWロニー 母国で試合中逮捕御嶽海苦言 朝乃山かわいそう速報全仏OP 錦織圭の1回戦蘭で主審がゴール 抗議実らず米大統領杯 来年以降も授与へ行くぞ G元木コーチの好判断頭真っ白 浜中ダービー初制覇J1神戸 ついに連敗脱出ロジャーバローズ ダービーV伊藤雅雪0-3 痛恨の王座陥落ドラ1近本 プロ初先頭打者弾座布団投げ処罰?厳戒の国技館サニブラウン 200日本歴代3位えぐい MLB投手らの佐々木評朝乃山 快勝も笑顔なき初VJ1神戸監督にべンゲル氏浮上ヤクルト2年ぶり悪夢の10連敗朝乃山 栃ノ心の一言に救われサーフィン五十嵐が日本人初V朝乃山 守った亡き恩師の教えメッセ9四死球 球審にボヤキ朝乃山初優勝 58年ぶりの快挙貴源治と貴ノ富士 双子同時VSBが米ドラ1右腕と契約合意巨人のスパイ?広島長野が心配頭部死球 一岡がG中島に謝罪香川 最終戦で鳴り響いた拍手錦織 世界30・40位以降が脅威画面にスカイウォーカー大谷アテネ金 鈴木桂治氏が復帰へ栃ノ心「判定負け」 抗議殺到38歳の鈴木桂治氏 現役復帰へ巨人戦で乱闘騒ぎ 警告試合にウグイス嬢 恒例の杉谷いじり栃ノ心ぼう然 勝ったと思った重量挙げ・三宅 涙の途中棄権異例 

In [19]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [106]:
wakati = " ".join(mecab_analysis(text))
# print(wakati)
mecab_analysis(text)

['平尾',
 '誠二',
 'さん',
 '娘',
 '父',
 'U',
 '20',
 '日本',
 '大人',
 'サッカー',
 '必要',
 '大谷',
 'MLB',
 '球宴',
 'DH',
 '候補',
 '入り',
 'クロ',
 'マー',
 '君',
 '観戦',
 '興奮',
 '松',
 '坂',
 '来月',
 '21',
 '日',
 '1',
 '軍',
 '復帰',
 '大坂',
 '初戦',
 '突破',
 '人生',
 '一番',
 '緊張',
 '仏',
 '大',
 '坂',
 '逆転',
 '初戦',
 '突破',
 '幻',
 '80',
 '年',
 '五輪',
 '代表',
 '笹田',
 '氏',
 '死去',
 '仏',
 'OP',
 '大坂',
 'なおみ',
 '1',
 '回戦',
 '八百長',
 'レアル',
 'DF',
 'ら',
 '逮捕',
 '浦和',
 '監督',
 '解任',
 '後任',
 '組長',
 '羽生',
 '結',
 '弦',
 '確か',
 '回復',
 '川内',
 '優輝',
 '世界',
 '陸上',
 '代表',
 '復帰',
 '青木',
 '宣',
 '親',
 '丸刈り',
 '体',
 '鼓舞',
 '中日',
 '松坂',
 '257',
 '日',
 'ぶり',
 '実戦',
 '登板',
 '188',
 'cm',
 '下手',
 'SB',
 '右',
 '腕相撲',
 '座布団',
 '罪',
 '?',
 '大',
 '坂',
 '赤土',
 '克服',
 '大谷',
 '技',
 '安打',
 '二',
 '盗',
 '失敗',
 '五輪',
 'チケ',
 '申し込み',
 'きょう',
 'ドラフト',
 '指名',
 '優先',
 '権',
 '変更',
 '2',
 '軍',
 '戦',
 '100',
 '球',
 '輝',
 '星',
 '昇格',
 '条件',
 '横審',
 '混乱',
 '審判',
 '部',
 '苦言',
 '豪',
 'V',
 'クビトバ',
 '仏',
 '棄権',
 '広島',
 '浮上',
 'きっかけ',
 '丸',
 '2',
 'ラン',
 '広島',
 '浮上',
 '4',
 '月

In [29]:
model =Word2Vec(res_list, size =100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [28]:
def calc_similarity(text1,text2):
    sim = model.similarity(text1,text2)
    return sim

In [34]:
calc_similarity("無限","アラート")

/home/t-katou/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.83386177

In [35]:
similar_word = model.most_similar(positive="ファン")
similar_word

/home/t-katou/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('分', 0.9897701740264893),
 ('提訴', 0.9889682531356812),
 ('機能', 0.9889172911643982),
 ('採用', 0.9888268113136292),
 ('万', 0.9888182878494263),
 ('物議', 0.9887837767601013),
 ('億', 0.988762617111206),
 ('8', 0.9887266159057617),
 ('3', 0.988698422908783),
 ('氏', 0.98869788646698)]

In [40]:
dictionary = Dictionary(res_list)
dictionary.filter_extremes(no_below=2,no_above=0.3)
print(dictionary.token2id)

{'2': 0, 'アラート': 1, '人': 2, '処分': 3, '無限': 4, '起訴': 5, 'NG': 6, 'TDS': 7, 'の': 8, '噴水': 9, '足': 10, 'エン': 11, 'タメ': 12, 'ポケモン': 13, '化': 14, '睡眠': 15, 'ゲーム': 16, '新作': 17, '本': 18, '発表': 19, 'DeNA': 20, '?': 21, 'スマホ': 22, '共感': 23, '海外': 24, '結婚式': 25, 'iPod': 26, 'touch': 27, '重点': 28, '新型': 29, '上位': 30, '商戦': 31, '異状': 32, '逆風': 33, 'コイン': 34, 'ビット': 35, '価格': 36, '回復': 37, '背景': 38, '一蹴': 39, '批判': 40, '日本国': 41, '氏': 42, '紀': 43, '見城': 44, '囚': 45, '林': 46, '死刑': 47, '理由': 48, '発信': 49, '長男': 50, '作家': 51, '幻冬舎': 52, '式': 53, '賞': 54, '贈呈': 55, 'コラ': 56, 'デマ': 57, '傘': 58, '拡散': 59, '画像': 60, '否定': 61, '大統領': 62, '招待': 63, '握手': 64, 'IT': 65, '外資': 66, '情報': 67, '流出': 68, '規制': 69, 'ロボット': 70, '声': 71, '導入': 72, '現場': 73, '配膳': 74, 'ハムスター': 75, '動画': 76, '虐待': 77, 'もろ刃': 78, '剣': 79, '排除': 80, '為': 81, '米': 82, '華': 83, 'ソフト': 84, '判定': 85, '商談': 86, '良しあし': 87, '不毛': 88, '不謹慎': 89, '連鎖': 90, '%': 91, '24': 92, '出荷': 93, '最大': 94, '減': 95, 'チャン': 96, 'パオ': 97, '休止': 98, '活動': 99,

In [41]:
vec = dictionary.doc2bow(res_list[0])
print(vec)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [44]:
dense = list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0])
print(dense)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [45]:
vec_dic = []
for i in res_list:
    vec = dictionary.doc2bow(i)
#     print(vec)
    vec_dic.append(list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]))

In [73]:
X = vec_dic
Y1 = ["0" for i in range(2000)]
Y2 = ["1" for i in range(2000)]
Y3 = ["2" for i in range(2000)]
Y4 = ["3" for i in range(2000)]
Y = Y1+Y2+Y3+Y4
# print(Y)
# print(Counter(Y))

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

In [42]:
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree


In [64]:
from sklearn.model_selection import GridSearchCV

In [75]:
param_grid ={"n_estimators":[50,100,200,300,400,500],
            "max_features":[1,3,10],
            "min_samples_split":[2,3,10],
            "min_samples_leaf":[1,3,10],
            "bootstrap":[True, False],
            "criterion":["gini","entropy"]
            }

forest_grid = GridSearchCV(estimator = ensemble.RandomForestClassifier(random_state=0),
                          param_grid = param_grid,
                          scoring="accuracy",
                          cv=3,
                          n_jobs=3)

forest_grid.fit(X_train,Y_train)

OSError: [Errno 22] Invalid argument

In [70]:
print(forest_grid.score)

OSError: [Errno 22] Invalid argument

In [74]:
random_forest=ensemble.RandomForestClassifier(max_depth=10)
random_forest.fit(X_train, Y_train)
random_forest.score(X_test,Y_test)

/home/t-katou/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.201875

In [ ]:
with closing(sqlite3.connect)